## Importing pandas inorder to read the json file

In [95]:
import pandas as pd

## With lines=True, Pandas will read each line as a separate record

In [96]:
data = pd.read_json('sales.json', lines=True)

In [97]:
data.head()

,_id,saleDate,items,storeLocation,customer,purchaseMethod
0,{'$oid': '5bd761dcae323e45a93ccfe8'},{'$date': '2015-03-23T21:06:49.506Z'},"[{'name': 'printer paper', 'tags': ['office', ...",Denver,"{'gender': 'M', 'age': 42, 'email': 'cauho@wit...",Online
1,{'$oid': '5bd761dcae323e45a93ccfe9'},{'$date': '2015-08-25T10:01:02.918Z'},"[{'name': 'envelopes', 'tags': ['stationary', ...",Seattle,"{'gender': 'M', 'age': 50, 'email': 'keecade@h...",Phone
2,{'$oid': '5bd761dcae323e45a93ccfea'},{'$date': '2017-06-22T09:54:14.185Z'},"[{'name': 'notepad', 'tags': ['office', 'writi...",Denver,"{'gender': 'M', 'age': 51, 'email': 'worbiduh@...",In store
3,{'$oid': '5bd761dcae323e45a93ccfeb'},{'$date': '2015-02-23T09:53:59.343Z'},"[{'name': 'binder', 'tags': ['school', 'genera...",Seattle,"{'gender': 'F', 'age': 45, 'email': 'vatires@t...",In store
4,{'$oid': '5bd761dcae323e45a93ccfec'},{'$date': '2017-12-03T18:39:48.253Z'},"[{'name': 'backpack', 'tags': ['school', 'trav...",London,"{'gender': 'M', 'age': 40, 'email': 'dotzu@ib....",In store


In [98]:
data.columns

Index(['_id', 'saleDate', 'items', 'storeLocation', 'customer',
       'purchaseMethod'],
      dtype='object')

In [99]:
data.dtypes 

_id               object
saleDate          object
items             object
storeLocation     object
customer          object
purchaseMethod    object
dtype: object

## Checking if there are any missing values (NaN)

In [100]:
print(data.isna().sum().sort_values())

_id               0
saleDate          0
items             0
storeLocation     0
customer          0
purchaseMethod    0
dtype: int64


## Helps Extracts and Transforms data from a nested JSON structure by creating  DataFrame where each row represents an individual item from a sales dataset, with associated sale and customer information.

In [101]:
# empty list to hold item data
item_data = []

#looped through each row
for index, row in data.iterrows():
    sale_id = row['_id']
    sale_date = row['saleDate']
    store_location = row['storeLocation']
    customer = row['customer']
    purchase_method = row['purchaseMethod']

    #loop through the 'items' list which contains the details about individual items sold.
    for item in row['items']:
        item_data.append({
            '_id': sale_id,
            'saleDate': sale_date,
            'storeLocation': store_location,
            'purchaseMethod': purchase_method,
            'quantity': item['quantity'],
            'price': item['price'],
            'name': item['name'],
            'gender': customer['gender'],
             
        })

df = pd.DataFrame(item_data)


In [102]:
df

,_id,saleDate,storeLocation,purchaseMethod,quantity,price,name,gender
0,{'$oid': '5bd761dcae323e45a93ccfe8'},{'$date': '2015-03-23T21:06:49.506Z'},Denver,Online,2,{'$numberDecimal': '40.01'},printer paper,M
1,{'$oid': '5bd761dcae323e45a93ccfe8'},{'$date': '2015-03-23T21:06:49.506Z'},Denver,Online,2,{'$numberDecimal': '35.29'},notepad,M
2,{'$oid': '5bd761dcae323e45a93ccfe8'},{'$date': '2015-03-23T21:06:49.506Z'},Denver,Online,5,{'$numberDecimal': '56.12'},pens,M
3,{'$oid': '5bd761dcae323e45a93ccfe8'},{'$date': '2015-03-23T21:06:49.506Z'},Denver,Online,2,{'$numberDecimal': '77.71'},backpack,M
4,{'$oid': '5bd761dcae323e45a93ccfe8'},{'$date': '2015-03-23T21:06:49.506Z'},Denver,Online,2,{'$numberDecimal': '18.47'},notepad,M
...,...,...,...,...,...,...,...,...
27433,{'$oid': '5bd761deae323e45a93ce36f'},{'$date': '2014-08-18T06:25:49.739Z'},New York,Online,5,{'$numberDecimal': '24.12'},envelopes,M
27434,{'$oid': '5bd761deae323e45a93ce36f'},{'$date': '2014-08-18T06:25:49.739Z'},New York,Online,4,{'$numberDecimal': '27.1'},pens,M
27435,{'$oid': '5bd761deae323e45a93ce36f'},{'$date': '2014-08-18T06:25:49.739Z'},New York,Online,3,{'$numberDecimal': '29.09'},binder,M
27436,{'$oid': '5bd761deae323e45a93ce36f'},{'$date': '2014-08-18T06:25:49.739Z'},New York,Online,2,{'$numberDecimal': '598.79'},laptop,M


In [103]:
df.dtypes 

_id               object
saleDate          object
storeLocation     object
purchaseMethod    object
quantity           int64
price             object
name              object
gender            object
dtype: object

## Show top 10 products (name) sales (quantity x price). 

## # Create a new dictionary where values are converted to floats


In [104]:
#convert values within dictionaries to float
def convert_to_float(lst):
    #empty list to store the modified dictionaries
    result = [dict([a, float(x)] for a, x in b.items()) for b in lst]
    return result

df['price'] = convert_to_float(df['price'])

## Define a function to extract the numeric value

In [105]:
#to extract the numeric value from the price dictionary
def extract_numeric_value(price_dict):
    return price_dict['$numberDecimal']

# Apply the function to the price
df['price'] = df['price'].apply(extract_numeric_value)
#calsulates sale
df['sales'] = df['price'] * df['quantity']

df.head()


,_id,saleDate,storeLocation,purchaseMethod,quantity,price,name,gender,sales
0,{'$oid': '5bd761dcae323e45a93ccfe8'},{'$date': '2015-03-23T21:06:49.506Z'},Denver,Online,2,40.01,printer paper,M,80.02
1,{'$oid': '5bd761dcae323e45a93ccfe8'},{'$date': '2015-03-23T21:06:49.506Z'},Denver,Online,2,35.29,notepad,M,70.58
2,{'$oid': '5bd761dcae323e45a93ccfe8'},{'$date': '2015-03-23T21:06:49.506Z'},Denver,Online,5,56.12,pens,M,280.60
3,{'$oid': '5bd761dcae323e45a93ccfe8'},{'$date': '2015-03-23T21:06:49.506Z'},Denver,Online,2,77.71,backpack,M,155.42
4,{'$oid': '5bd761dcae323e45a93ccfe8'},{'$date': '2015-03-23T21:06:49.506Z'},Denver,Online,2,18.47,notepad,M,36.94


In [106]:
#Groupby 'name', calculate the sum of 'sales' for each product, and reset the index
top_products = df.groupby('name')['sales'].sum().reset_index()
# Sort top_productsin descending order based on the sales column
top_products = top_products.sort_values(by='sales', ascending=False)

# Select the top 10 products
top_10_products = top_products.head(10)
print(top_10_products)
#print unique product name
print(df['name'].unique())


            name       sales
3         laptop  6775977.07
0       backpack   817374.10
5           pens   581843.27
1         binder   511644.57
4        notepad   463615.48
2      envelopes   376658.49
6  printer paper   367459.29
['printer paper' 'notepad' 'pens' 'backpack' 'envelopes' 'binder' 'laptop']


## Show top 3 products (name) sales by store (location). 

In [107]:
# Group the data by 'name' and 'storeLocation,' and calculate total sales
grouped = df.groupby(['name', 'storeLocation'])['sales'].sum().reset_index()

# Sort the data within each store location group to find the top 3 products
top_products = grouped.groupby('storeLocation').apply(lambda x: x.nlargest(3, 'sales')).reset_index(drop=True)

print(top_products)

        name storeLocation       sales
0     laptop        Austin  1018494.05
1   backpack        Austin   115217.59
2       pens        Austin    74622.52
3     laptop        Denver  1961659.25
4   backpack        Denver   245679.34
5       pens        Denver   185008.39
6     laptop        London  1073098.72
7   backpack        London   144529.14
8       pens        London    92602.30
9     laptop      New York   694613.32
10  backpack      New York    82563.14
11      pens      New York    64029.90
12    laptop     San Diego   462973.58
13  backpack     San Diego    50997.42
14      pens     San Diego    42134.55
15    laptop       Seattle  1565138.15
16  backpack       Seattle   178387.47
17      pens       Seattle   123445.61


## Show rankings of each store (location). 

In [108]:
# Group by store location, sum sales, and rank
store_rankings = df.groupby('storeLocation')['sales'].sum().reset_index()
# Calculate the rank of each store location based on their total sales
store_rankings['rank'] = store_rankings['sales'].rank(ascending=False, method='min')
print(store_rankings.sort_values('rank'))


  storeLocation       sales  rank
1        Denver  2921009.92   1.0
5       Seattle  2255947.69   2.0
2        London  1583066.79   3.0
0        Austin  1445603.11   4.0
3      New York  1016059.59   5.0
4     San Diego   672885.17   6.0


## Show purchased method by gender table 

In [112]:
# Group by gender and payment method, count the purchases
purchase_table = df.groupby(['gender', 'purchaseMethod']).size().unstack()
purchase_table = pd.DataFrame(purchase_table)
purchase_table

purchaseMethod,In store,Online,Phone
gender,,,
F,8062,4270,1499
M,7571,4314,1722


## Show monthly total sales 

In [110]:
# Define a function to extract the numeric value
def extract_numeric_value(price_dict):
    return price_dict['$date']

# Apply the function to the saleDate
df['saleDate'] = df['saleDate'].apply(extract_numeric_value)

df.head()

,_id,saleDate,storeLocation,purchaseMethod,quantity,price,name,gender,sales
0,{'$oid': '5bd761dcae323e45a93ccfe8'},2015-03-23T21:06:49.506Z,Denver,Online,2,40.01,printer paper,M,80.02
1,{'$oid': '5bd761dcae323e45a93ccfe8'},2015-03-23T21:06:49.506Z,Denver,Online,2,35.29,notepad,M,70.58
2,{'$oid': '5bd761dcae323e45a93ccfe8'},2015-03-23T21:06:49.506Z,Denver,Online,5,56.12,pens,M,280.60
3,{'$oid': '5bd761dcae323e45a93ccfe8'},2015-03-23T21:06:49.506Z,Denver,Online,2,77.71,backpack,M,155.42
4,{'$oid': '5bd761dcae323e45a93ccfe8'},2015-03-23T21:06:49.506Z,Denver,Online,2,18.47,notepad,M,36.94


In [111]:
# Convert the saleDate column to datetime , handling errors with coerce
df['saleDate'] = pd.to_datetime(df['saleDate'], format='%Y-%m-%dT%H:%M:%S.%fZ', errors='coerce')

# Format the dates as month and create a new column
df['month'] = df['saleDate'].dt.strftime("%m")
#contains the total sales for each month
monthly_sales = df.groupby('month')['sales'].sum().reset_index() 
print(monthly_sales)


   month      sales
0     01  918003.43
1     02  733884.21
2     03  867736.82
3     04  756559.46
4     05  878892.77
5     06  769416.01
6     07  847051.30
7     08  791504.16
8     09  758566.50
9     10  838232.29
10    11  860402.92
11    12  871436.60
